In [1]:
import boto3
from IPython.display import clear_output


AWS_REGION = "sa-east-1"
STREAM_NAME = "taxi-trips-stream"

In [2]:
client = boto3.client(
    'kinesis',
    region_name=AWS_REGION
)

In [3]:
client.list_streams()

{'StreamNames': ['taxi-trips-stream'],
 'HasMoreStreams': False,
 'ResponseMetadata': {'RequestId': 'd80098f8-6860-bf16-8f44-36c10e242c66',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd80098f8-6860-bf16-8f44-36c10e242c66',
   'x-amz-id-2': 'WalTFK4fcRoIrJRzIg6C9VF+mTcpctanQkLlxhdd/bKnZvA/I1yrIbyGFzXyeos2MmWS7Ng1BXgdrcud0y4sSvWsX3JMSt94',
   'date': 'Sun, 24 May 2020 17:16:41 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '60'},
  'RetryAttempts': 0}}

In [ ]:
from multiprocess.pool import ThreadPool
import time

files = [
    '../data/raw/data-sample_data-nyctaxi-trips-2009-json_corrigido.json',
    '../data/raw/data-sample_data-nyctaxi-trips-2010-json_corrigido.json',
    '../data/raw/data-sample_data-nyctaxi-trips-2011-json_corrigido.json',
    '../data/raw/data-sample_data-nyctaxi-trips-2012-json_corrigido.json'
]

max_bytes = 215000 # 215 kb 
total_sent = 0

def send_register(registers):
    global total_sent
    clear_output(wait=True)
    
    length = len(registers)
    total_sent += length
    
    print(f"Sending {length} registers to Kinesis.")
    records = [
        {
            'Data': data.encode(),
            'PartitionKey': '1'
        }
        for data in registers
    ]
    print(f"Total: {total_sent}")
    client.put_records(
        Records=records,
        StreamName=STREAM_NAME,
    )

def process_file(file):
    with open(file, 'r') as buffer:
        registers = buffer.readlines(max_bytes)
        while len(registers):
            send_register(registers)
            registers = buffer.readlines(max_bytes)

pool = ThreadPool(4)
pool.map(process_file, files)